In [ ]:
import bionumpy as bnp
from pyjaspar import jaspardb
from bionumpy.sequence.position_weight_matrix import PWM, get_motif_scores
import numpy as np
import plotly.express as plx

Get the ctcf bed file<br>
! wget https://www.encodeproject.org/files/ENCFF843VHC/@@download/ENCFF843VHC.bed.gz

In [ ]:
peaks = bnp.open("ENCFF843VHC.bed.gz").read()
reference_genome = bnp.open_indexed("/home/knut/Data/hg38.fa")
contig_lenghts = reference_genome.get_contig_lengths()
sorted_peaks = bnp.arithmetics.sort_all_intervals(peaks, sort_order=list(contig_lenghts.keys()))

contig_lenghts = bnp.MultiStream.sort_dict_by_key(reference_genome.get_contig_lengths())

We need to synch our peaks to the reference genome

In [ ]:
multistream = bnp.MultiStream(contig_lenghts,
                              intervals=sorted_peaks,
                              sequence=reference_genome)

Get the sequences of the peaks

In [ ]:
sequence_stream = bnp.sequence.get_sequences(multistream.sequence,
                                             multistream.intervals)

Merge all the sequences from the stream

In [ ]:
sequences = np.concatenate(list(sequence_stream))

Get all human motifs :)

In [ ]:
jdb_obj = jaspardb(release="JASPAR2020")
human_motifs = jdb_obj.fetch_motifs(collection="CORE", species=["9606"])

In [ ]:
counts = []
lengths = []
# Calculate the motif scores for each sequence
for i, motif in enumerate(human_motifs):
    if i % 10 == 0:
        print(i)
    pwm = PWM.from_dict(motif.pwm)
    motif_scores = get_motif_scores(sequences, pwm)
    adjusted_scores = motif_scores-np.logaddexp(motif_scores, motif.length*np.log(0.25))
    counts.append((adjusted_scores.max(axis=-1) > np.log(0.9)).sum())
    lengths.append(motif.length)

heck that lentghts are not an influencing factor

In [ ]:
plx.scatter(lengths, counts)

In [ ]:
names = [m.name for m in human_motifs]

ort by number of significant hit

In [ ]:
args = np.argsort(counts)
sorted_names = [names[i] for i in args]